# Modelo 5

Para entrenar a los algoritmos se tiene en cuenta:

- Edad
- Genero (por rangos de 5)
- Educacion (estado maximo de todos los estudios)
- Areas de trabajo

In [4]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split

# Cargo CSV

In [5]:
test_final = pd.read_csv('../../data/test_final_100k.csv')
test_final = test_final.drop(columns='id')

In [6]:
postulantes_educacion = pd.read_csv('../../data_sin_duplicados/fiuba_1_postulantes_educacion.csv')
postulantes_educacion = postulantes_educacion.drop(columns='Unnamed: 0')
postulantes_genero_y_edad = pd.read_csv('../../data_sin_duplicados/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.drop(columns='Unnamed: 0')
#vistas = pd.read_csv('../../data_sin_duplicados/fiuba_3_vistas.csv')
#vistas = vistas.drop(columns='Unnamed: 0')
postulaciones = pd.read_csv('../../data_sin_duplicados/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.drop(columns='Unnamed: 0')
avisos_online = pd.read_csv('../../data_sin_duplicados/fiuba_5_avisos_online.csv')
avisos_online = avisos_online.drop(columns='Unnamed: 0')
avisos_detalle = pd.read_csv('../../data_sin_duplicados/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.drop(columns='Unnamed: 0')

# Procesamiento de datos

##### Agrego edad

In [7]:
postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'],errors='coerce')
postulantes_genero_y_edad['edad'] = datetime.datetime.now().year - postulantes_genero_y_edad['fechanacimiento'].dt.year

In [8]:
# Despues veo que hago con los que no tienen edad
postulantes_genero_y_edad.isna().sum()

idpostulante           0
fechanacimiento    26096
sexo                   0
edad               26096
dtype: int64

##### Separo datos

In [9]:
# GENTE NO POSTULADA
gente_no_postulada = postulantes_genero_y_edad[-postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_no_postulada = gente_no_postulada.reset_index(drop=True)

In [10]:
# GENTE POSTULADA
gente_postulada = postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_postulada = gente_postulada.reset_index(drop=True)

##### Genero

In [11]:
gente_no_postulada['sexo'].value_counts()

FEM           71992
MASC          67819
NO_DECLARA    15707
0.0               6
Name: sexo, dtype: int64

In [12]:
# Pocos datos con sexo invalido, los desecho ya que son los no postulados
gente_no_postulada = gente_no_postulada.drop(gente_no_postulada[gente_no_postulada['sexo'] == '0.0'].index)

In [13]:
gente_postulada['sexo'].value_counts()

FEM           179213
MASC          160034
NO_DECLARA      9635
0.0                1
Name: sexo, dtype: int64

In [14]:
# Reemplazo el sexo invalido por NO_DECLARA
gente_postulada.loc[gente_postulada['sexo'] == '0.0','sexo'] = 'NO_DECLARA'

##### Edad (cont.)

In [15]:
# Dropeo los datos invalidos de los no postulados (se podria buscar alguna forma de salvarlos)
gente_no_postulada = gente_no_postulada.dropna()
gente_no_postulada = gente_no_postulada.reset_index(drop=True)
gente_no_postulada['edad'] = gente_no_postulada['edad'].astype('int32')

In [16]:
gente_no_postulada.isna().sum()

idpostulante       0
fechanacimiento    0
sexo               0
edad               0
dtype: int64

In [17]:
# En un principio los relleno con la media de los postulantes
gente_postulada.loc[gente_postulada['edad'].isna(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())
gente_postulada['edad'] = gente_postulada['edad'].astype('int32')

In [18]:
gente_postulada.isna().sum()

idpostulante           0
fechanacimiento    10074
sexo                   0
edad                   0
dtype: int64

##### Educacion

In [19]:
def educacion_ponderado_por_estado_de_estudios(postulantes_educacion):
    dic = {'Graduado': 3,'En Curso': 2,'Abandonado': 1}
    postulantes_educacion['estado'] = postulantes_educacion['estado'].map(dic)
    postulantes_educacion = postulantes_educacion.groupby(['idpostulante','nombre']).max().unstack(fill_value=0)
    postulantes_educacion.columns = postulantes_educacion.columns.droplevel()
    postulantes_educacion = postulantes_educacion.reset_index()
    postulantes_educacion = postulantes_educacion.rename_axis('',axis='columns')
    return postulantes_educacion

In [20]:
postulantes_educacion = educacion_ponderado_por_estado_de_estudios(postulantes_educacion)

In [21]:
postulantes_educacion.head()

,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,0z5Dmrd,0,0,0,0,0,0,3
1,0z5JW1r,0,0,3,0,3,3,1
2,0z5VvGv,0,0,0,0,3,0,0
3,0zB01pE,0,0,0,0,0,3,1
4,0zB026d,0,0,0,0,3,3,0


##### Vistas

In [22]:
avisos_detalle = avisos_detalle.drop(['ciudad','mapacalle','nombre_zona','descripcion','titulo','tipo_de_trabajo','nivel_laboral','denominacion_empresa','idpais'],axis=1)

In [23]:
replacements = {
   'nombre_area': {
      'Ingeniería Geológica|Ingeniería Metalúrgica|Ingeniería  Metalurgica|Ingeniería  Eléctrica y Electrónica|Ingeniería  Automotriz|Ingeniería de Producto|Ingeniería de Procesos|Ingeniería de Ventas|Ingeniería Oficina Técnica / Proyecto|Ingeniería en Alimentos|Ingeniería Agrónoma|Ingeniería Química|Ingeniería  Industrial|Ingeniería Electromecánica|Ingeniería  Mecánica|Otras Ingenierias': 'Ingenieria',      
      'Corporate Finance / Banca Inversión|Evaluación Económica|Organización y Métodos|Cuentas Corrientes|Cadetería|Contabilidad|Impuestos|Auditoría|Control de Gestión|Consultoria': 'Administración',#cuentasc podria ir en ventas
      'Diseño Multimedia|Diseño 3D|Diseño Textil e Indumentaria|Diseño Web|Diseño de Interiores / Decoración|Diseño Industrial|Diseño Gráfico': 'Diseño y Urbanismo',
      'Periodismo|Comunicaciones Internas|Relaciones Institucionales/Publicas|Responsabilidad Social': 'Comunicacion',
      'Auditoría de Seguros|Venta de Seguros|Tecnico de Seguros|Administración de Seguros|Administracion de Seguros': 'Seguros',
      'Quimica|Bioquimica|Bioquímica': 'Química',
      'Selección|Capacitación|Compensación y Planilla|Administración de Personal': 'Recursos Humanos', #administracion de personal dudo
      'Exploración Minera y Petroquimica|Ingeniería en Petróleo y Petroquímica|Salud Minera|Seguridad Industrial|Medio Ambiente|Instrumentación Minera|Ingeniería en Minas|Mineria/Petroleo/Gas': 'Minería',
      'Traduccion': 'Idiomas',
      'Mantenimiento|Calidad|Programación de producción': 'Producción',
      'Inversiones / Proyectos de Inversión|Finanzas Internacionales|Análisis de Riesgos|Tesorería|Créditos y Cobranzas|Facturación|Planeamiento económico-financiero': 'Finanzas',
      'Telefonista|Call Center|Telemarketing|Caja': 'Atención al Cliente', #no se si caja va aca
      'Almacén / Depósito / Expedición|Compras|Distribución|Abastecimiento' : 'Logística',
      'Legal|Asesoría Legal Internacional': 'Legales',
      'Sistemas|Telecomunicaciones|Tecnologia/Sistemas de producción|Oficios Informática|Tecnologías de la Información|Data Warehousing|Redes|Data Entry|Administración de Base de Datos|Seguridad Informática|Soporte Técnico|Testing / QA / QC|Internet|Infraestructura|Análisis Funcional|Liderazgo de Proyecto|Programación' : 'Tecnologia/Sistemas',
      'Oficios y Profesiones|Producción y Limpieza|Estética y Cuidado Personal|Mantenimiento y Limpieza|Seguridad|Servicios|Promotoras/es|Jóvenes Profesionales': 'Oficios',#aca tambien
      'Recepcionista|Asistente': 'Secretaria',
      'Dirección de Obra|Arquitectura|Ingeniería Civil|Urbanismo|Operaciones|Diseño y Construcción|Topografía' : 'Construcción',
      'Compras Internacionales/Importación|Ventas Internacionales/Exportación|Consultorías Comercio Exterior|Apoderado Aduanal|Logística Internacionales/Importación': 'Comercio Exterior',
      'Business Intelligence|E-commerce|Community Management|Producto|Media Planning|Mercadotecnia Internacional': 'Marketing',
      'Camareros|Hotelería|Turismo': 'Gastronomia',
      'Seguridad Industrial|Oficios e Higiene' : 'Seguridad e Higiene',
      'Comercial|Ventas económico-financiero|Planeamiento Comercial|Planeamiento|Desarrollo de Negocios|Negocios Internacionales|Ventas comercial' : 'Ventas',
      'Laboratorio|Farmacia industrial|Farmacia comercial|Medicina|Otras Especialidades médicas|Diagnóstico por imágenes|Esterilización|Maternidad|Farmacéutica|Veterinaria|Odontología|Farmacia hospitalaria|Farmacia Industrial|Farmacia Comercial|Medicina Laboral|Instrumentación|Auditoría Médica|Emergentología|Kinesiología / terapia ocupacional|Prácticas cardiológicas|Otras áreas técnicas en salud|Fonoaudiología|Clínica Médica|Cuidados Intensivos Cardiologicos Adultos|Psicología|Instrumentación quirúrgica|Administración Médica' : 'Salud',
      'Gerencia / Dirección General|Dirección': 'Gerencia',
      'Pasantía / Trainee|Prácticas Profesionales': 'Pasantía',
      'Tecnologia / Tecnologia/Sistemas': 'Tecnologia/Sistemas',
      'Diseño y Construcción':'Construcción',
      'Secretaria de Tráfico': 'Logística',
      'Ventas comercial':'Ventas',
      'Salud Laboral|Salud quirúrgica':'Salud',
      'Educación/ Docentes|Educación especial|Trabajo social|Trabajo Social|Investigación y Desarrollo|Bienestar Estudiantil': 'Educación',
      'Arte y Cultura|Comunicaciones Externas|Independientes|Transporte|Creatividad|Multimedia|Back Office': 'Otros' #transporte entraria tanto en logistica como en oficios y creatividad en marketing y diseño
       #Multimedia en marketing y diseño. Back Office puede entrar en muchos
   }
}

avisos_detalle.replace(replacements, regex=True, inplace=True)

In [24]:
avisos_detalle.loc[avisos_detalle['nombre_area'] =='Enfermería (ver Enfermería)', "nombre_area"] = "Salud"

# Funciones

In [25]:
# Fijarse de balancear el sample con 50% de postulados y no postulados
# Recibe las series de idpostulante e idaviso
def generador_de_sample_serie(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = pd.DataFrame(idpostulante)
    idaviso = pd.DataFrame(idaviso)
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop(columns='tem')
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [26]:
def borrar_tuplas_test_del_sample(sample,test_final):
    df_return = sample.merge(test_final,how='left',on=['idaviso','idpostulante'],indicator=True)
    df_return = df_return[df_return['_merge'] == 'left_only']
    df_return = df_return.drop(columns='_merge')
    return df_return

In [27]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

In [28]:
def genero_edad(tupla_idaviso_idpostulante,postulantes_genero_y_edad):
    df_return = tupla_idaviso_idpostulante.merge(postulantes_genero_y_edad,on='idpostulante',how='left')
    df_return = df_return.drop(columns='fechanacimiento')
    return df_return

In [29]:
def educacion(tupla_idaviso_idpostulante):
    df_return = tupla_idaviso_idpostulante.merge(postulantes_educacion,on='idpostulante',how='left')
    return df_return

In [30]:
def edad_en_rangos(tupla_idaviso_idpostulante):
    dic = {'(10, 15]':0 , '(15, 20]':1 , '(20, 25]':2 , '(25, 30]':3 , '(30, 35]':4 , '(35, 40]':5 , '(40, 45]':6 , '(45, 50]':7 , '(50, 55]':8 , '(55, 60]':9 , '(60, 65]':10 , '(65, 70]':11 , '(70, 75]':12 , '(75, 80]':13 , '(80, 85]':14 , '(85, 90]':15 , '(95, 100]':16 , '(240, 245]':17}
    tupla_idaviso_idpostulante['edad'] = pd.cut(tupla_idaviso_idpostulante['edad'], range(0, 100, 5))
    tupla_idaviso_idpostulante['edad'] = tupla_idaviso_idpostulante['edad'].astype(str)
    tupla_idaviso_idpostulante['edad'] = tupla_idaviso_idpostulante['edad'].map(dic)

In [31]:
def area_de_trabajo(tupla_a_predecir):
    df_return = tupla_a_predecir.merge(avisos_detalle,on='idaviso',how='left')
    return df_return

# Gente postulada

In [32]:
tupla_postulada = postulaciones.drop(columns='fechapostulacion')
tupla_postulada['y'] = 1
tupla_postulada = genero_edad(tupla_postulada,gente_postulada)
tupla_postulada = oneHotEncoding(tupla_postulada,'sexo')
tupla_postulada = educacion(tupla_postulada)

In [33]:
tupla_postulada.iloc[:,7:] = tupla_postulada.iloc[:,7:].fillna(0).astype('int32')

In [34]:
tupla_postulada.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,1112257047,NM5M,1,48,1,0,0,0,0,0,0,3,0,0
1,1111920714,NM5M,1,48,1,0,0,0,0,0,0,3,0,0
2,1112346945,NM5M,1,48,1,0,0,0,0,0,0,3,0,0
3,1112345547,NM5M,1,48,1,0,0,0,0,0,0,3,0,0
4,1112237522,5awk,1,56,1,0,0,0,0,0,0,0,3,3


# Gente no postulada

In [35]:
sample = generador_de_sample_serie(avisos_detalle['idaviso'],gente_no_postulada['idpostulante'],100,200)
sample['y'] = 0
sample = genero_edad(sample,gente_no_postulada)
sample = oneHotEncoding(sample,'sexo')
sample = educacion(sample)

In [36]:
sample = borrar_tuplas_test_del_sample(sample,test_final)

In [37]:
sample.iloc[:,7:] = sample.iloc[:,7:].fillna(0).astype('int32')

In [38]:
sample.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,1112390041,GNZ8JWK,0,55,0,1,0,0,0,0,0,0,0,0
1,1112390041,ZDlz5E1,0,21,1,0,0,0,0,0,0,3,0,2
2,1112390041,a0N4q,0,43,1,0,0,0,0,0,0,0,0,3
3,1112390041,ep41j9,0,32,0,1,0,0,0,3,0,0,0,2
4,1112390041,awOL1m,0,34,1,0,0,0,0,0,3,0,0,3


# Test Final

In [39]:
test_final = genero_edad(test_final,gente_postulada.append(gente_no_postulada))

In [40]:
# Veo si hay datos nulos
test_final.isna().sum()

idaviso            0
idpostulante       0
sexo            2278
edad            2278
dtype: int64

In [41]:
# Salvo datos nulos
test_final.loc[test_final['sexo'].isna(),'sexo'] = 'NO_DECLARA'
test_final.loc[test_final['edad'].isna(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())

In [42]:
test_final.isna().sum()

idaviso         0
idpostulante    0
sexo            0
edad            0
dtype: int64

In [43]:
test_final['edad'] = test_final['edad'].astype('int32')
test_final = oneHotEncoding(test_final,'sexo')
test_final = educacion(test_final)

In [44]:
test_final.isna().sum()

idaviso                 0
idpostulante            0
edad                    0
FEM                     0
MASC                    0
NO_DECLARA              0
Doctorado            5612
Master               5612
Otro                 5612
Posgrado             5612
Secundario           5612
Terciario/Técnico    5612
Universitario        5612
dtype: int64

In [45]:
# Salvo educacion nula
test_final.iloc[:,6:] = test_final.iloc[:,6:].fillna(0).astype('int32')

In [46]:
test_final.isna().sum()

idaviso              0
idpostulante         0
edad                 0
FEM                  0
MASC                 0
NO_DECLARA           0
Doctorado            0
Master               0
Otro                 0
Posgrado             0
Secundario           0
Terciario/Técnico    0
Universitario        0
dtype: int64

In [47]:
edad_en_rangos(test_final)

In [48]:
test_final.isna().sum()

idaviso              0
idpostulante         0
edad                 5
FEM                  0
MASC                 0
NO_DECLARA           0
Doctorado            0
Master               0
Otro                 0
Posgrado             0
Secundario           0
Terciario/Técnico    0
Universitario        0
dtype: int64

In [49]:
test_final['edad'] = test_final['edad'].fillna(2)

In [50]:
test_final.isna().sum()

idaviso              0
idpostulante         0
edad                 0
FEM                  0
MASC                 0
NO_DECLARA           0
Doctorado            0
Master               0
Otro                 0
Posgrado             0
Secundario           0
Terciario/Técnico    0
Universitario        0
dtype: int64

In [51]:
# Agrego area de trabajo
test_final = area_de_trabajo(test_final)
test_final = oneHotEncoding(test_final,'nombre_area')

In [52]:
test_final = test_final.drop(columns=['idaviso','idpostulante'])

In [53]:
test_final.head()

,edad,FEM,MASC,NO_DECLARA,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,...,Recursos Humanos,Salud,Secretaria,Secretaria de Tráfico,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Tecnologia/Sistemas de producción,Ventas,Ventas económico-financiero
0,6.0,1,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,4.0,0,1,0,0,0,3,0,0,3,...,0,0,0,0,0,0,0,0,1,0
2,5.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,11.0,0,1,0,0,0,0,3,0,0,...,0,0,0,0,0,0,1,0,0,0
4,4.0,1,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,1,0,0,0


# ML

In [55]:
sample.shape

(1999987, 14)

In [56]:
del postulantes_genero_y_edad
del postulaciones
del avisos_online

In [57]:
# Metodo para calcular el error de la competencia
from sklearn.metrics import roc_auc_score

In [58]:
sample = sample.append(tupla_postulada.sample(n=2000000,random_state=0))

In [59]:
# Separo la edad en rangos
edad_en_rangos(sample)
sample['edad'] = sample['edad'].fillna(2)

In [60]:
sample.isna().sum()

idaviso              0
idpostulante         0
y                    0
edad                 0
FEM                  0
MASC                 0
NO_DECLARA           0
Doctorado            0
Master               0
Otro                 0
Posgrado             0
Secundario           0
Terciario/Técnico    0
Universitario        0
dtype: int64

In [61]:
sample.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,1112390041,GNZ8JWK,0,8.0,0,1,0,0,0,0,0,0,0,0
1,1112390041,ZDlz5E1,0,2.0,1,0,0,0,0,0,0,3,0,2
2,1112390041,a0N4q,0,6.0,1,0,0,0,0,0,0,0,0,3
3,1112390041,ep41j9,0,4.0,0,1,0,0,0,3,0,0,0,2
4,1112390041,awOL1m,0,4.0,1,0,0,0,0,0,3,0,0,3


In [62]:
sample.tail()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
3486559,1112371232,bO4k2W4,1,4.0,0,1,0,0,0,0,0,3,0,0
2143356,1112313300,5mqQJVv,1,2.0,1,0,0,0,0,3,0,3,2,0
2463467,1112387552,5lGdvp,1,5.0,1,0,0,0,0,0,0,3,3,2
3125407,1112444102,0zkAOmr,1,1.0,1,0,0,0,0,0,0,3,0,2
1570387,1112345051,6RG42v,1,4.0,1,0,0,0,0,0,0,3,3,0


In [63]:
# Agrego area de trabajo
sample = area_de_trabajo(sample)
sample = oneHotEncoding(sample,'nombre_area')

In [64]:
sample = sample.drop(columns=['idaviso','idpostulante'])

In [65]:
len(sample)

3999987

In [66]:
sample.head()

,y,edad,FEM,MASC,NO_DECLARA,Doctorado,Master,Otro,Posgrado,Secundario,...,Recursos Humanos,Salud,Secretaria,Secretaria de Tráfico,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Tecnologia/Sistemas de producción,Ventas,Ventas económico-financiero
0,0,8.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2.0,1,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
2,0,6.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,4.0,0,1,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,4.0,1,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
# Exporto modelo
#sample.to_csv('modelo_5_datos_2m.csv')
# Cargo modelo
#sample = pd.read_csv('modelo_1_datos_2m.csv')
#sample = sample.drop(columns='Unnamed: 0')
#sample.head()

In [70]:
# Divido el set
X_train, X_test, y_train, y_test = train_test_split(sample.drop(columns='y'),sample['y'],random_state=0,test_size=0.01)

In [71]:
del sample
del avisos_detalle
del gente_postulada
del gente_no_postulada

# Perceptron

In [72]:
from sklearn.linear_model import Perceptron

In [73]:
model = Perceptron(penalty='l1')
model.fit(X_train,y_train)
predict = model.predict(X_test)

/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [74]:
roc_auc_score(y_test,predict)

0.5821979126066641

# KNN

In [ ]:
#from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#model = KNeighborsClassifier(n_neighbors=3)
#model.fit(X_train,y_train)
#predict = model.predict(X_test)

In [ ]:
#roc_auc_score(y_test,predict)

# Random Forest

In [75]:
from sklearn.ensemble import RandomForestClassifier

In [76]:
model = RandomForestClassifier(n_estimators=100,random_state=0, n_jobs=-1)
model.fit(X_train, y_train)
predict = model.predict(X_test)

In [77]:
roc_auc_score(y_test,predict)

0.7338538096664382

# Gradient Boosting Classifier

In [81]:
from sklearn.ensemble import GradientBoostingClassifier

In [82]:
model = GradientBoostingClassifier(n_estimators=150)
model.fit(X_train, y_train)
predict = model.predict(X_test)

In [83]:
roc_auc_score(y_test,predict)

0.6976553664994346

In [78]:
predict = model.predict(test_final)

In [79]:
predict = pd.DataFrame(predict,columns=['sepostulo'])

In [80]:
predict.to_csv('modelo_5_datos_4m_RDF')